# *FLAML test*
Testa FLAML på travet

In [ ]:
import pandas as pd 
import numpy as np 
from IPython.display import display 

df = pd.read_csv('..\\all_data.csv')   

In [ ]:
# prepare
PLAC_MEAN=df.plac.mean()
# Set a smooth mean value to  features in df
def calc_smooth_mean(df, by, y, m=300, tot_mean=PLAC_MEAN):
    # print(by, y, m)
    # Compute the number of values and the mean of each group
    agg = df.groupby(by)[y].agg(['count', 'mean'])
    counts = agg['count']
    means = agg['mean']

    # Compute the "smoothed" means
    smooth = (counts * means + (m * tot_mean)) / (counts + m)
    # print('used m = ',m)
    return smooth.to_dict()

def transf_kusk_häst(df,pref='',m=300,the_map=None):
    
    df[pref+'ekipage'] = df[pref+ 'kusk'].str.cat(df['häst'], sep =", ")  # concatenate 'häst' and 'kusk' into one column
    # print('skapade ekipage', df[pref+'ekipage'].shape)
    
    if not the_map:
        the_map = calc_smooth_mean(df, by=pref+'ekipage', y='plac',m=50) # make numeric with Target encoding with smooth mean
        
    df['ekipage'].map(the_map) 
    # print('ekipage klart')
        
    df.drop([pref+'kusk'],axis=1,inplace=True)
    return the_map, df

the_map,df = transf_kusk_häst(df,pref='', m=300)
_,df = transf_kusk_häst(df,pref='h1_', m=300, the_map=the_map)
_,df = transf_kusk_häst(df,pref='h2_', m=300, the_map=the_map)
_,df = transf_kusk_häst(df,pref='h3_', m=300, the_map=the_map)
_,df = transf_kusk_häst(df,pref='h4_', m=300, the_map=the_map)
_,df = transf_kusk_häst(df,pref='h5_', m=300, the_map=the_map)

df.columns

In [ ]:
proc=0.75
alla_datum = list(df.datum.unique())
split_ix = int(len(alla_datum)*proc)
y_train=df[split_ix:]['plac']
y_test =df[:split_ix]['plac']
y_train = (y_train==1)*1
y_test  = (y_test==1)*1
X_train = df[split_ix:].copy()
X_test  = df[:split_ix].copy()
X_train.drop(['avd','startnr','h1_dat','h2_dat','h3_dat','h4_dat','h5_dat', 'vodds','podds','dist','bins'   ,'plac'],axis=1,inplace=True)
X_test.drop (['avd','startnr','h1_dat','h2_dat','h3_dat','h4_dat','h5_dat', 'vodds','podds','dist','bins'   ,'plac'],axis=1,inplace=True)
X_train

X_train.shape,X_test.shape,y_train.shape,y_test.shape

In [ ]:
# FLAML test
from flaml import AutoML 
automl = AutoML()
automl.fit(X_train, y_train,task = 'classification',split_type=None, time_budget=3000, n_jobs=5, early_stop=True, ensemble=True)

In [ ]:
display(automl.model.get_params())
display(automl.model)
display(automl.best_estimator)

train_pred = automl.predict_proba(X_train)
test_pred = automl.predict_proba(X_test)
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.metrics import plot_confusion_matrix, confusion_matrix


In [ ]:
print('train auc',roc_auc_score(y_train, train_pred[:,1]))
print('test auc',roc_auc_score(y_test, test_pred[:,1]))
res_train = pd.DataFrame()
res_test  = pd.DataFrame()
res_train['y_train'] = y_train
res_train['pred_train'] = train_pred[:,1]
res_test['y_test'] = y_test
res_test['pred_test'] = test_pred[:,1]

display(f'mean for train=1 {res_train.loc[res_train.y_train==1].pred_train.mean()}')
display(f'mean for train=0 {res_train.loc[res_train.y_train==0].pred_train.mean()}')
display(f'mean for test==1 {res_test.loc[res_test.y_test==1].pred_test.mean()}')
display(f'mean for test==0 {res_test.loc[res_test.y_test==0].pred_test.mean()}')

mätningar = pd.DataFrame()
mätningar['batch'] = [5,4,3,2,1]
mätningar['train_AUC'] = [0.8345, 0.8352,0.8290, 0.8291, 0.8311]
mätningar['test_AUC'] = [0.8418,0.8380,0.8368, 0.8356, 0.8360]
mätningar['avg train win'] =[0.2342,0.2381,0.2267, 0.2241, 0.2258]
mätningar['avg test win'] =[0.2472,0.2377,0.2394, 0.2365, 0.2364]

mätningar


In [ ]:
# automl.model.model.get_feature_importance(prettified=True)
automl.model.model.estimators[0][1].model.get_feature_importance(prettified=True)


In [ ]:
import pickle
filename = 'FLAML_model.sav'
pickle.dump(automl.model, open(filename, 'wb'))
